In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

YEARS = set()

table = dict()
table["population"]  = pd.read_csv("data/population.csv")
table["pcgdp"]       = pd.read_csv("data/pcgdp.csv")
table["gini"]        = pd.read_csv("data/gini.csv")
table["electricity"] = pd.read_csv("data/electricity.csv")
table["life_exp"]    = pd.read_csv("data/life_exp.csv")
winter_raw = pd.read_csv("data/winter.csv")



DROP_FROM = 1960
DROP_TO = 1985
years_to_drop = [str(i) for i in range(DROP_FROM, DROP_TO)]
for key, _ in table.items():
    print(key)
    table[key] = table[key].drop(columns=years_to_drop+[
        "Country Name", "Indicator Name", "Indicator Code",
        "Unnamed: 62", "2017"])
    table[key] = table[key].rename(index=str, columns={"Country Code":"Country"})

population
pcgdp
gini
electricity
life_exp


In [2]:
table["population"].head(3)

,Country,1985,1986,1987,1988,1989,1990,1991,1992,1993,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,ABW,63026.0,62644.0,61833.0,61079.0,61032.0,62149.0,64622.0,68235.0,72504.0,...,101220.0,101353.0,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0
1,AFG,11783050.0,11601041.0,11502761.0,11540888.0,11777609.0,12249114.0,12993657.0,13981231.0,15095099.0,...,26616792.0,27294031.0,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0
2,AGO,10609042.0,10921037.0,11218268.0,11513968.0,11827237.0,12171441.0,12553446.0,12968345.0,13403734.0,...,20997687.0,21759420.0,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0


In [3]:
table["population"].describe()

KeyboardInterrupt: 

In [ ]:
table["pcgdp"].head(3)

In [ ]:
table["pcgdp"].describe()

In [ ]:
table["electricity"].head(3)

In [ ]:
table["electricity"].describe()

In [ ]:
winter_raw.head(5)

In [ ]:
medals = dict() # by countries code => by years


for i in range(len(winter_raw)):
    row = dict(winter_raw.iloc[i, :])
    ccode = row["Country"]
    year  = row["Year"]
    medal = row["Medal"]
    if int(year) < DROP_TO:
        continue
        
    YEARS.add(year)
    medals.setdefault(ccode, dict()).setdefault(year, 0)
    medals[ccode][year] += 1
print("total country medalists", len(medals))

In [ ]:
COUNTRIES = pd.DataFrame(table["population"]["Country"])
def get_w_r():
    winter = COUNTRIES
    for i, year in enumerate(YEARS):
        winter[year] = np.zeros(len(winter))
        for ccode, _ in medals.items():
            medals[ccode].setdefault(year, 0)
            winter.loc[winter.Country==ccode, year] = medals[ccode][year]
    return winter
winter = get_w_r()

In [ ]:
winter.head(5)
a = []
for i in range(len(winter)):
    row = winter.iloc[i]
    if row.drop(columns="Country").all():
        print(row.Country)

In [ ]:
winter.loc[winter.Country=="RUS"]

In [ ]:
for key, _ in table.items():
    table[key] = table[key].sort_values(by=["Country"]).drop(columns=["Country"])
    table[key] = table[key].fillna(table[key].mean()).fillna(0)
    std = table[key][str(year)].std()
    table[key][str(year)] = (table[key][str(year)] - table[key][str(year)].mean()) / std
    #table[key] /= table[key].max() - table[key].min()
    
winter = winter.sort_values(by=["Country"]).drop(columns=["Country"])
winter = winter.fillna(winter.mean()).fillna(0)
#std = winter[year].std()
#winter[year] = (winter[year] - winter[year].mean()) / std
#winter /= winter.max() - winter.min()
winter /= winter.sum(axis=0)

In [ ]:
winter.head(5)

In [ ]:
winter.iloc[0]
for key, val in table.items():
    table[key].to_csv(key+"_edited.csv")
YEARS = sorted(list(YEARS))

In [ ]:
X  = [] # population, pcgdp normed in year = year
y  = [] # medals
y2 = [] # classification, if have medals

feature_names = ["population", "pcgdp", "electricity", "gini", "life_exp"]
#feature_names = ["population", "pcgdp", "life_exp"]
mean_year_diff = 7

def prev_year(year):
    if YEARS.index(year):
        return YEARS[YEARS.index(year)-1]
    else:
        return year - 4

def split(test_year, regression=True):
    X_train, y_train, X_test, y_test = [], [], [], []
    y2_train, y2_test = [], []
    for year_i, year in enumerate(YEARS):
        start = year - mean_year_diff + 1
        years_ = [str(i) for i in range(start, year + 1)]
        
        for i in range(len(winter)):    
            if start < min(YEARS):
                continue
            features = []
            for feature in feature_names:
                a = np.array(table[feature][years_].iloc[i])
                features.append(a.mean())
            features.append(winter[prev_year(year)].iloc[i])

            target = float(winter[year].iloc[i])
            if not (year in test_year):
                X_train.append(features)
                y_train.append(target)
                y2_train.append(target > 0)
            else:
                X_test.append(features)
                y_test.append(target)
                y2_test.append(target > 0)
                
    if not regression:
        y_train, y_test = y2_train, y2_test
        
    return np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)

In [ ]:
X, y, _, _ = split([2020], True)
_, y2, _, _ = split([2020], False)
print(len(X), len(y), len(y2))

In [ ]:
def get_only_positive(X, y):
    print(len(X))
    X_, y_ = [], []
    for i in range(len(y)):
        if y[i] <= 0: 
            continue
        X_.append(X[i])
        y_.append(y[i])
    return np.array(X_), np.array(y_)

X_, y_ = get_only_positive(X, y)
print(len(y_))

def oversample(X, y):
    print(len(X))
    X_, y_ = [], []
    cnt=0
    for i in range(len(y)):
        if y[i] <= 0: 
            cnt+=1
        X_.append(X[i])
        y_.append(y[i])
    mult = len(y) // (len(y) - cnt) - 1
    for i in range(len(y)):
        nowm = 0
        if y[i] >= 0: 
            nowm = mult
        for j in range(nowm):
            X_.append(X[i])
            y_.append(y[i])
    return np.array(X_), np.array(y_)


In [ ]:
from sklearn import datasets, linear_model, ensemble
from sklearn.model_selection import cross_val_score

def test(X_, y_, X, y2):
    models_reg = {"linreg" : linear_model.LinearRegression(),
                 "ridge": linear_model.Ridge(),
                 "huber": linear_model.HuberRegressor(),
                 "gb" : ensemble.GradientBoostingRegressor()}

    X_train, y_train = X_, y_

    for k, m in models_reg.items():
        print(k, cross_val_score(m, X_train, y_train, cv=10).mean()) 


    models_clf = {"logreg" : linear_model.LogisticRegression(),
                 "ridge": linear_model.RidgeClassifier(),
                 "rf": ensemble.RandomForestClassifier(),
                 "sgd": linear_model.SGDClassifier(),
                 "gb" : ensemble.GradientBoostingClassifier()}

    X_train, y_train = X, y2

    for k, m in models_clf.items():
        print(k, cross_val_score(m, X_train, y_train, cv=10).mean())


test(X_, y_, X, y2)

In [ ]:
def test2(X_train, y_train, X_test, y_test, reg=True):
    models = {"linreg" : linear_model.LinearRegression(),
                 "ridge": linear_model.Ridge(),
                 "huber": linear_model.HuberRegressor(),
                 "rf" : ensemble.RandomForestRegressor(),
                 "gb" : ensemble.GradientBoostingRegressor()}
    models_clf = {"logreg" : linear_model.LogisticRegression(),
                 "ridge": linear_model.RidgeClassifier(),
                 "rf": ensemble.RandomForestClassifier(),
                 "sgd": linear_model.SGDClassifier(),
                 "gb" : ensemble.GradientBoostingClassifier()}
    
    if not reg:
        models = models_clf

    for k, m in models.items():
        m = m.fit(X_train, y_train)
        print(k, m.score(X_test, y_test))
        if (k == 'rf'):
            print(m.feature_importances_)


# Regression over counties with medals
print(len(X))
X_train, y_train, X_test, y_test = split([2006], True)
X_train, y_train = oversample(X_train, y_train)
X_test, y_test = oversample(X_test, y_test)
print(len(X_train), len(y_train), len(X_test), len(y_test))
test2(X_train, y_train, X_test, y_test, True)
# classification over all (>0 medals)
X_train, y_train, X_test, y_test = split([2006], False)
test2(X_train, y_train, X_test, y_test, False)

In [ ]:
winter.hist()

In [ ]:
for year in YEARS:
    print(year)
    print(len(winter.loc[winter[year] > 0]) / len(winter))

In [ ]:
X_train, y_train, X_test, y_test = split([2014], True)
X_train, y_train = get_only_positive(X_train, y_train)
X_test, y_test = get_only_positive(X_test, y_test)
m = ensemble.GradientBoostingRegressor(n_estimators=30, max_depth=3).fit(X_train, y_train)
y_predicted = m.predict(X_test)


plt.scatter(y_test, y_predicted)
plt.title('Scatter plot pythonspot.com')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()
print(m.score(X_test, y_test))
# only countries with medals

In [ ]:
from sklearn.metrics import r2_score

winter_old = get_w_r()
def year_predict(year):
    X_train, y_train, X_test, y_test = split([year], True)
    X_train, y_train = oversample(X_train, y_train)
    mR = linear_model.Ridge().fit(X_train, y_train)

    X_train, y_train, _, _ = split([year], False)
    X_train, y_train = oversample(X_train, y_train)

    m = ensemble.RandomForestClassifier().fit(X_train, y_train)
    y_predicted = m.predict_proba(X_test)


    THRESHOLD = 0.40
    y_pred = np.zeros(len(y_predicted))
    for i in range(len(y_predicted)):
        y_pred[i] = float(y_predicted[i][1] > THRESHOLD)
        if y_pred[i] > 0:
            y_pred[i] = mR.predict([X_test[i]])[0]

    sum_mult = float(winter_old[year].sum())
    plt.scatter(y_test*sum_mult, y_pred*sum_mult, alpha=0.3)
    plt.title('Scatter plot '  + str(year))
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.show()
    print(r2_score(y_test, y_pred))

In [ ]:
year_predict([2006])

In [ ]:
year_predict([2010])

In [ ]:
year_predict([2014])